<a href="https://colab.research.google.com/github/genkai10/Capstone_roughflow/blob/main/Copy_of_FallDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import h5py
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.initializers import Constant
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from google.colab import drive
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

drive.mount('/content/drive')

# Function to extract features from video frames
def extract_features_from_video(video_path, target_size=(64, 64)):
    cap = cv2.VideoCapture(video_path)
    features = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Resize frame
        frame = cv2.resize(frame, target_size)
        # Normalize pixel values
        frame = frame / 255.0
        features.append(frame)
    cap.release()
    return features

# Function to load dataset
def load_dataset(data_dir, max_frames=100):
    X = []
    y = []
    for class_name in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, class_name)
        if os.path.isdir(class_dir):
            for video_file in os.listdir(class_dir):
                if video_file.endswith('.mp4'):  # Assuming videos are in .mp4 format
                    video_path = os.path.join(class_dir, video_file)
                    features = extract_features_from_video(video_path)
                    # Pad or truncate the sequences to max_frames
                    if len(features) < max_frames:
                        features.extend([np.zeros_like(features[0])] * (max_frames - len(features)))
                    elif len(features) > max_frames:
                        features = features[:max_frames]
                    X.append(features)
                    y.append(class_name)
    return np.array(X), np.array(y)

# Navigate to the directory containing the dataset
os.chdir("/content/drive/MyDrive/URFD/URFD-Augmented")

# Load dataset
X, y = load_dataset("/content/drive/MyDrive/URFD/URFD-Augmented")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
X_train, X_test, y_train, y_test

(array([[[[[0.36078431, 0.36078431, 0.36078431],
           [0.42352941, 0.42352941, 0.42352941],
           [0.42352941, 0.42352941, 0.42352941],
           ...,
           [0.5372549 , 0.57647059, 0.69411765],
           [0.14901961, 0.16862745, 0.29019608],
           [0.27058824, 0.24313725, 0.28235294]],
 
          [[0.35294118, 0.35294118, 0.35294118],
           [0.43529412, 0.43529412, 0.43529412],
           [0.43921569, 0.43921569, 0.43921569],
           ...,
           [0.54509804, 0.55294118, 0.71372549],
           [0.23529412, 0.29411765, 0.37647059],
           [0.4627451 , 0.40784314, 0.4745098 ]],
 
          [[0.34901961, 0.34901961, 0.34901961],
           [0.43529412, 0.43529412, 0.43529412],
           [0.44705882, 0.44705882, 0.44705882],
           ...,
           [0.17647059, 0.18431373, 0.31764706],
           [0.21568627, 0.35294118, 0.39607843],
           [0.38823529, 0.38039216, 0.43529412]],
 
          ...,
 
          [[0.03921569, 0.03921569, 0.039215

In [6]:
model = Sequential()
model.add(layers.Conv3D(32, (3, 3, 3), activation='relu', input_shape=(100, 64, 64, 3), bias_initializer=Constant(0.01)))
model.add(layers.Conv3D(32, (3, 3, 3), activation='relu', bias_initializer=Constant(0.01)))
model.add(layers.MaxPooling3D((2, 2, 2)))
model.add(layers.Conv3D(64, (3, 3, 3), activation='relu'))
model.add(layers.Conv3D(64, (2, 2, 2), activation='relu'))
model.add(layers.MaxPooling3D((2, 2, 2)))
model.add(layers.Dropout(0.6))
model.add(layers.Flatten())
model.add(layers.Dense(256, 'relu'))
model.add(layers.Dropout(0.7))
model.add(layers.Dense(128, 'relu'))
model.add(layers.Dropout(0.5))
#model.add(layers.Dense(10, 'softmax'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 98, 62, 62, 32)    2624      
                                                                 
 conv3d_1 (Conv3D)           (None, 96, 60, 60, 32)    27680     
                                                                 
 max_pooling3d (MaxPooling3  (None, 48, 30, 30, 32)    0         
 D)                                                              
                                                                 
 conv3d_2 (Conv3D)           (None, 46, 28, 28, 64)    55360     
                                                                 
 conv3d_3 (Conv3D)           (None, 45, 27, 27, 64)    32832     
                                                                 
 max_pooling3d_1 (MaxPoolin  (None, 22, 13, 13, 64)    0         
 g3D)                                                   

In [7]:
model.compile(Adam(0.001), 'binary_crossentropy', ['accuracy'])
model.fit(X_train,y_train,epochs=10,batch_size=32,verbose=1,validation_data=(X_test,y_test),callbacks=[EarlyStopping(patience=15)])
#Testing the 3D-CNN
_, acc = model.evaluate(X_train, y_train)
print('training accuracy:', str(round(acc*100, 2))+'%')
_, acc = model.evaluate(X_test, y_test)
print('testing accuracy:', str(round(acc*100, 2))+'%')

Epoch 1/10
4/4 [==============================] - 271s 65s/step - loss: 1.1727 - accuracy: 0.5089 - val_loss: 0.7064 - val_accuracy: 0.5357
Epoch 2/10
4/4 [==============================] - 269s 65s/step - loss: 0.6926 - accuracy: 0.5982 - val_loss: 0.6855 - val_accuracy: 0.5357
Epoch 3/10
4/4 [==============================] - 269s 65s/step - loss: 0.6773 - accuracy: 0.6250 - val_loss: 0.6782 - val_accuracy: 0.5714
Epoch 4/10
4/4 [==============================] - 267s 64s/step - loss: 0.6611 - accuracy: 0.6339 - val_loss: 0.6446 - val_accuracy: 0.6071
Epoch 5/10
4/4 [==============================] - 267s 64s/step - loss: 0.6471 - accuracy: 0.5982 - val_loss: 0.6150 - val_accuracy: 0.9643
Epoch 6/10
4/4 [==============================] - 267s 65s/step - loss: 0.6268 - accuracy: 0.6607 - val_loss: 0.5134 - val_accuracy: 1.0000
Epoch 7/10
4/4 [==============================] - 266s 64s/step - loss: 0.5074 - accuracy: 0.8125 - val_loss: 0.3366 - val_accuracy: 1.0000
Epoch 8/10
4/4 [====